# Prediction
Load the model, intialize the speakers with 1 audio for each person, then predict the speaker of unseen audios among the previous identified speakers

In [1]:
from experiments.predict import *
from voicemap.train import TrainingArgs
from IPython.display import Audio, display

/home/profenpoche/voicemap/voicemap/datasets/librispeech.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
args = TrainingArgs()
args.model_path = "../saved_models/gaixoa.pt"
args.dim = 1
args.filters = 128
args.num_speakers = 1291
args.batch_size = 1500
args.n_seconds = 3
args.downsampling = 4
args.spectrogram = False
args.window_length = 0.02
args.window_hop = 0.01
args.device = 'cuda'

In [3]:
#############
# Constants #
#############
# Device must be 'cpu' if no gpu found
device = torch.device(args.device)
if args.spectrogram:
    if args.dim == 1:
        in_channels = int(args.window_length * 16000) // 2 + 1
    elif args.dim == 2:
        in_channels = 1
    else:
        raise RuntimeError
else:
    in_channels = 1
num_classes = args.num_speakers

N_SPEAKERS_EVALUATION = 5

In [4]:
def getAudioFromPredictionDataloader(predictionDataloader):
    if args.spectrogram:
        return predictionDataloader.dataset.dataset[0][0]
    else:
        return predictionDataloader.dataset[0][0]

## Load model

In [5]:
model = load_model(args.model_path, args.filters, in_channels, num_classes, args.dim, args.device)
model

Model loaded : ../saved_models/gaixoa.pt


ResidualClassifier(
  (conv1): Conv1d(1, 128, kernel_size=(7,), stride=(2,), bias=False)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock1D(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock1D(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affin

## Initialisation phase

In [6]:
n_members = 4
auto_prepare = int(input("Do you want to auto prepare "+str(n_members)+" speakers ? (0/1) : "))
if (auto_prepare):
    audio_paths = [
        "/home/profenpoche/voicemap/data/testPrediction/activity_SId/1-10.flac", 
        "/home/profenpoche/voicemap/data/testPrediction/activity_SId/2-18.flac", 
        #"/home/profenpoche/voicemap/data/testPrediction/activity_SId/3-18.flac", 
        #"/home/profenpoche/voicemap/data/testPrediction/activity_SId/6-4.flac",
        "/home/profenpoche/voicemap/data/testPrediction/activity_SId/7-2.flac",
        "/home/profenpoche/voicemap/data/testPrediction/activity_SId/8-4.flac"
        # "data/testPrediction/activity_SId/4-1.flac", 
        # "data/testPrediction/activity_SId/5-1.flac"
        ]
else:
    n_members = int(input("Please enter the number of speakers in the activity : "))

Do you want to auto prepare 4 speakers ? (0/1) : 1


In [7]:
# Initialize the list of identified and trusted speakers
speakers = [] 
audios = []
# First step is to associate speakers with audio file foreach member of the group
for m in range(n_members):
    if auto_prepare:
        audio_path = audio_paths[m]
    else:
        audio_path = str(input("Please enter the audio path for the voice of the speaker n°"+str(m)+" : "))

    # It is necessary to put the audio in a dataloader before prediction
    predictionDataloader = buildPredictionDataloader(audio_path, args=args)
    # predict the speaker (and validate him compared to the previous speakers)
    speaker_identified = identifySpeaker(speakers, predictionDataloader, model, N_SPEAKERS_EVALUATION, args.spectrogram)
    audio = getAudioFromPredictionDataloader(predictionDataloader)
    audios.append(audio)
    display(Audio(data=audio, rate=4000))
    speakers.append(speaker_identified)

print(bcolors.OKBLUE + "\nSpeakers are : ")
print(speakers)
print(bcolors.ENDC)


-------------- Prediction of an audio : /home/profenpoche/voicemap/data/testPrediction/activity_SId/1-10.flac
Predicted : 
962			531			897			189			77
7.509284358641514	7.169433977042695	5.5788167055619855	5.268428187180964	3.8458224246544894
--- 0.22578144073486328 seconds ---



-------------- Prediction of an audio : /home/profenpoche/voicemap/data/testPrediction/activity_SId/2-18.flac
Predicted : 
756			306			5			835			796
9.7385295563469	4.6379370512772145	4.221647777000394	3.8979450629220542	3.7989043452648588
--- 0.19322919845581055 seconds ---



-------------- Prediction of an audio : /home/profenpoche/voicemap/data/testPrediction/activity_SId/7-2.flac
Predicted : 
716			870			710			403			548
5.440760488494652	3.4142579395313035	3.132485441201555	3.0195222205418895	2.9130790475510375
--- 0.1723928451538086 seconds ---



-------------- Prediction of an audio : /home/profenpoche/voicemap/data/testPrediction/activity_SId/8-4.flac
Predicted : 
688			1169			581			1146			445
4.126651864760496	4.12556590970538	4.087121474062838	3.753236740491122	3.35008139984941
--- 0.17401456832885742 seconds ---



Speakers are : 
[962, 756, 716, 688]



## Prediction phase

In [ ]:
# Then we can loop and ask for an unlimited number of audio file
# Foreach audio we can link the most confident speaker among the 'speakers' list
activity_can_continue = True
while activity_can_continue:
    try:
        audio_path = str(input("Please enter the audio path for the speaker to identify : "))
        predictionDataloader = buildPredictionDataloader(audio_path, args)
        
        audio = getAudioFromPredictionDataloader(predictionDataloader)
        print(f"{bcolors.OKBLUE} Audio you have entered : {bcolors.ENDC}")
        display(Audio(data=audio, rate=4000))
        
        values, indices = predictAudioFile(predictionDataloader, model, N_SPEAKERS_EVALUATION,args.spectrogram)
        
        print("\nConfidence for each speaker previously identified : ")
        filtered_list = [values[i] for i in speakers]
        print(filtered_list)
        speaker_index = filtered_list.index(max(filtered_list))
        
        print(bcolors.BOLD + "====== Speaker identified is : "+str(speaker_index) + bcolors.ENDC)
        display(Audio(data=audios[speaker_index], rate=4000))
        print("########################################################")
    except NameError as error:
        print(error)
        activity_can_continue = bool(input("NameError. Press Enter to leave "))
    except RuntimeError as error:
        print(error)
        activity_can_continue = bool(input("RuntimeError. Press Enter to leave "))


Please enter the audio path for the speaker to identify : /home/profenpoche/voicemap/data/testPrediction/activity_SId/8-4.flac
 Audio you have entered : 



-------------- Prediction of an audio : /home/profenpoche/voicemap/data/testPrediction/activity_SId/8-4.flac
Predicted : 
688			1169			581			1146			1128
4.071573234445161	3.9869047687436305	3.730469617742817	3.7298819548291107	3.4452308942185335

Confidence for each speaker previously identified : 
[0.4126253032679354, -0.24203997790405285, 0.11934282427426199, 4.071573234445161]
====== Speaker identified is : 3


########################################################
